# FAISS Vector Database Testing for Healthcare Test Case Generation

This notebook demonstrates the use of FAISS (Facebook AI Similarity Search) for building a vector database to store and retrieve healthcare test cases. This is foundational for the AI-powered test case generation system as outlined in the NASSCOM problem statement.

## Key Objectives:
1. Create sample healthcare test cases and requirements
2. Convert text to embeddings using Sentence Transformers
3. Build and test a FAISS vector index
4. Perform similarity searches to find relevant test cases
5. Demonstrate how this can support automated test case generation


In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer
import json
from typing import List, Dict, Any
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")
print(f"FAISS version: {faiss.__version__ if hasattr(faiss, '__version__') else 'Unknown'}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")


✅ Libraries imported successfully!
FAISS version: 1.12.0
NumPy version: 2.3.3
Pandas version: 2.3.2


## Step 1: Initialize Embedding Model

We'll use Sentence Transformers to convert text into embeddings. The model `all-MiniLM-L6-v2` is lightweight yet effective for semantic similarity tasks.


In [2]:
# Initialize the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Test the model with a sample healthcare text
sample_text = "Patient blood pressure monitoring test case"
sample_embedding = model.encode(sample_text)

print(f"✅ Model loaded successfully!")
print(f"Embedding dimension: {len(sample_embedding)}")
print(f"Sample embedding (first 10 values): {sample_embedding[:10]}")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Model loaded successfully!
Embedding dimension: 384
Sample embedding (first 10 values): [-0.09741534  0.10009975 -0.05815928 -0.0040833   0.02157647 -0.00060688
  0.04464454 -0.01896917 -0.00650744 -0.05049402]


## Step 2: Create Sample Healthcare Test Cases

Let's create a diverse set of healthcare test cases that cover various aspects of medical software testing, including:
- Patient data management
- Medical device integration
- Clinical workflows
- Compliance requirements
- Emergency scenarios


In [3]:
# Create sample healthcare test cases
healthcare_test_cases = [
    {
        "id": "TC001",
        "title": "Patient Registration with Complete Demographics",
        "description": "Verify that a new patient can be registered with all required demographic information including name, DOB, address, insurance, and emergency contact",
        "category": "Patient Management",
        "priority": "High",
        "compliance": ["HIPAA", "GDPR"],
        "test_steps": "1. Navigate to patient registration\n2. Enter all required fields\n3. Validate insurance\n4. Save patient record",
        "expected_result": "Patient record created successfully with unique MRN assigned"
    },
    {
        "id": "TC002",
        "title": "Blood Pressure Monitor Integration Test",
        "description": "Test integration with Omron blood pressure monitor device for automatic vital signs capture",
        "category": "Medical Device Integration",
        "priority": "Critical",
        "compliance": ["FDA 510(k)", "ISO 13485"],
        "test_steps": "1. Connect BP monitor via Bluetooth\n2. Take patient reading\n3. Verify automatic data transfer\n4. Check data accuracy",
        "expected_result": "BP readings automatically transferred and stored in patient chart within 5 seconds"
    },
    {
        "id": "TC003",
        "title": "Medication Allergy Alert System",
        "description": "Verify that system generates appropriate alerts when prescribing medications that patient is allergic to",
        "category": "Clinical Decision Support",
        "priority": "Critical",
        "compliance": ["FDA", "ISO 27001"],
        "test_steps": "1. Add penicillin allergy to patient record\n2. Attempt to prescribe amoxicillin\n3. Verify alert appears\n4. Test override with reason",
        "expected_result": "System displays critical allergy alert and requires documented reason for override"
    },
    {
        "id": "TC004",
        "title": "HL7 Lab Results Import",
        "description": "Test importing laboratory results via HL7 v2.5 message format from external lab system",
        "category": "Interoperability",
        "priority": "High",
        "compliance": ["HL7 Standards", "CLIA"],
        "test_steps": "1. Send HL7 ORU message\n2. Verify message parsing\n3. Check result mapping\n4. Confirm physician notification",
        "expected_result": "Lab results correctly parsed, stored, and physician notified of critical values"
    },
    {
        "id": "TC005",
        "title": "Emergency Department Triage Workflow",
        "description": "Test complete ED triage workflow including ESI level assignment and vital signs documentation",
        "category": "Clinical Workflow",
        "priority": "Critical",
        "compliance": ["EMTALA", "Joint Commission"],
        "test_steps": "1. Register walk-in patient\n2. Complete triage assessment\n3. Assign ESI level\n4. Document vital signs\n5. Queue for provider",
        "expected_result": "Patient triaged within 10 minutes with appropriate ESI level and queued correctly"
    },
    {
        "id": "TC006",
        "title": "GDPR Data Export Request",
        "description": "Verify system can export all patient data in machine-readable format for GDPR compliance",
        "category": "Compliance",
        "priority": "High",
        "compliance": ["GDPR", "Data Protection"],
        "test_steps": "1. Submit data export request\n2. Verify all data categories included\n3. Check export format\n4. Validate completeness",
        "expected_result": "Complete patient data exported in JSON/XML format within 30 days"
    },
    {
        "id": "TC007",
        "title": "Radiology PACS Image Viewer",
        "description": "Test viewing and manipulating DICOM images from PACS system",
        "category": "Imaging",
        "priority": "High",
        "compliance": ["DICOM", "FDA"],
        "test_steps": "1. Open patient imaging study\n2. Test zoom/pan/window level\n3. Add measurements\n4. Create report",
        "expected_result": "Images load within 3 seconds with full manipulation capabilities"
    },
    {
        "id": "TC008",
        "title": "Prescription Drug Monitoring Integration",
        "description": "Verify integration with state PDMP for controlled substance monitoring",
        "category": "Pharmacy",
        "priority": "Critical",
        "compliance": ["DEA", "State Regulations"],
        "test_steps": "1. Prescribe controlled substance\n2. Query PDMP database\n3. Review patient history\n4. Document review",
        "expected_result": "PDMP query completed and documented before prescription finalized"
    },
    {
        "id": "TC009",
        "title": "Telemedicine Video Consultation",
        "description": "Test end-to-end telemedicine visit including video, audio, and documentation",
        "category": "Telehealth",
        "priority": "High",
        "compliance": ["HIPAA", "State Licensure"],
        "test_steps": "1. Schedule virtual visit\n2. Test video/audio quality\n3. Complete consultation\n4. Generate visit summary",
        "expected_result": "Successful encrypted video consultation with complete documentation"
    },
    {
        "id": "TC010",
        "title": "Clinical Trial Enrollment Screening",
        "description": "Test automated screening for clinical trial eligibility based on patient criteria",
        "category": "Research",
        "priority": "Medium",
        "compliance": ["GCP", "21 CFR Part 11"],
        "test_steps": "1. Define trial criteria\n2. Run eligibility query\n3. Review matched patients\n4. Generate recruitment list",
        "expected_result": "System identifies all eligible patients meeting inclusion/exclusion criteria"
    },
    {
        "id": "TC011",
        "title": "Insulin Pump Data Integration",
        "description": "Test integration with Medtronic insulin pump for diabetes management",
        "category": "Medical Device Integration",
        "priority": "High",
        "compliance": ["FDA Class II", "ISO 13485"],
        "test_steps": "1. Connect insulin pump\n2. Download pump data\n3. Analyze glucose patterns\n4. Generate recommendations",
        "expected_result": "Pump data successfully imported with trend analysis and recommendations"
    },
    {
        "id": "TC012",
        "title": "Audit Trail for Controlled Substances",
        "description": "Verify complete audit trail for all controlled substance prescriptions",
        "category": "Compliance",
        "priority": "Critical",
        "compliance": ["DEA", "21 CFR Part 11"],
        "test_steps": "1. Prescribe controlled substance\n2. Modify prescription\n3. Cancel prescription\n4. Review audit log",
        "expected_result": "All actions logged with timestamp, user, and reason for change"
    },
    {
        "id": "TC013",
        "title": "Sepsis Alert Algorithm",
        "description": "Test early warning system for sepsis detection based on vital signs and lab values",
        "category": "Clinical Decision Support",
        "priority": "Critical",
        "compliance": ["CMS SEP-1", "Clinical Guidelines"],
        "test_steps": "1. Enter abnormal vitals\n2. Add lab results\n3. Verify alert triggers\n4. Test response workflow",
        "expected_result": "Sepsis alert triggered within 1 hour of meeting criteria with bundled orders"
    },
    {
        "id": "TC014",
        "title": "Patient Portal Secure Messaging",
        "description": "Test secure messaging between patients and providers through patient portal",
        "category": "Patient Engagement",
        "priority": "Medium",
        "compliance": ["HIPAA", "Meaningful Use"],
        "test_steps": "1. Patient sends message\n2. Provider receives notification\n3. Provider responds\n4. Check encryption",
        "expected_result": "Messages encrypted end-to-end with delivery confirmation"
    },
    {
        "id": "TC015",
        "title": "Vaccine Administration Record",
        "description": "Test complete vaccine administration workflow including inventory and reporting",
        "category": "Immunization",
        "priority": "High",
        "compliance": ["CDC", "State Registry"],
        "test_steps": "1. Check vaccine inventory\n2. Document administration\n3. Update immunization record\n4. Report to state registry",
        "expected_result": "Vaccine documented and reported to state registry within 24 hours"
    }
]

# Convert to DataFrame for easier manipulation
df_test_cases = pd.DataFrame(healthcare_test_cases)
print(f"✅ Created {len(df_test_cases)} healthcare test cases")
print(f"\nCategories: {df_test_cases['category'].unique()}")
print(f"\nPriorities: {df_test_cases['priority'].value_counts().to_dict()}")
print(f"\nSample test case:\n{df_test_cases.iloc[0][['id', 'title', 'category']].to_dict()}")


✅ Created 15 healthcare test cases

Categories: ['Patient Management' 'Medical Device Integration'
 'Clinical Decision Support' 'Interoperability' 'Clinical Workflow'
 'Compliance' 'Imaging' 'Pharmacy' 'Telehealth' 'Research'
 'Patient Engagement' 'Immunization']

Priorities: {'High': 7, 'Critical': 6, 'Medium': 2}

Sample test case:
{'id': 'TC001', 'title': 'Patient Registration with Complete Demographics', 'category': 'Patient Management'}


## Step 3: Generate Embeddings for Test Cases

We'll create embeddings by combining multiple fields to capture the full context of each test case.


In [4]:
# Create combined text for each test case for better embedding quality
def create_test_case_text(row):
    """Combine relevant fields to create comprehensive text representation"""
    compliance_str = ', '.join(row['compliance']) if isinstance(row['compliance'], list) else row['compliance']
    
    text = f"""
    Test Case: {row['title']}
    Category: {row['category']}
    Priority: {row['priority']}
    Description: {row['description']}
    Test Steps: {row['test_steps']}
    Expected Result: {row['expected_result']}
    Compliance: {compliance_str}
    """
    return text.strip()

# Generate combined text for all test cases
df_test_cases['combined_text'] = df_test_cases.apply(create_test_case_text, axis=1)

# Generate embeddings for all test cases
print("Generating embeddings for all test cases...")
embeddings = model.encode(df_test_cases['combined_text'].tolist(), show_progress_bar=True)

print(f"✅ Generated embeddings with shape: {embeddings.shape}")
print(f"Embedding dimension: {embeddings.shape[1]}")


Generating embeddings for all test cases...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Generated embeddings with shape: (15, 384)
Embedding dimension: 384


## Step 4: Build FAISS Index

We'll create different types of FAISS indices and compare their performance:


In [5]:
# Build FAISS indices

# 1. Flat Index (Exact Search) - Most accurate but slower for large datasets
dimension = embeddings.shape[1]
index_flat = faiss.IndexFlatL2(dimension)
index_flat.add(embeddings.astype('float32'))

print(f"✅ Flat Index created with {index_flat.ntotal} vectors")

# 2. IVF Index (Inverted File Index) - Faster search with slight accuracy tradeoff
nlist = 5  # Number of clusters (small dataset, so few clusters)
quantizer = faiss.IndexFlatL2(dimension)
index_ivf = faiss.IndexIVFFlat(quantizer, dimension, nlist)

# Train the index with our data
index_ivf.train(embeddings.astype('float32'))
index_ivf.add(embeddings.astype('float32'))

print(f"✅ IVF Index created with {index_ivf.ntotal} vectors")

# 3. HNSW Index (Hierarchical Navigable Small World) - Good balance of speed and accuracy
index_hnsw = faiss.IndexHNSWFlat(dimension, 32)  # 32 is the connectivity parameter
index_hnsw.add(embeddings.astype('float32'))

print(f"✅ HNSW Index created with {index_hnsw.ntotal} vectors")

print(f"\n📊 Index Comparison:")
print(f"  - Flat Index: Exact search, O(n) complexity")
print(f"  - IVF Index: Approximate search, faster for large datasets")
print(f"  - HNSW Index: Graph-based search, good balance")


✅ Flat Index created with 15 vectors
✅ IVF Index created with 15 vectors
✅ HNSW Index created with 15 vectors

📊 Index Comparison:
  - Flat Index: Exact search, O(n) complexity
  - IVF Index: Approximate search, faster for large datasets
  - HNSW Index: Graph-based search, good balance


WARNING clustering 15 points to 5 centroids: please provide at least 195 training points


## Step 5: Create Search Function

Let's create a reusable search function that can find similar test cases based on queries:


In [6]:
def search_similar_test_cases(query: str, index: faiss.Index, k: int = 5, df: pd.DataFrame = df_test_cases):
    """
    Search for similar test cases using FAISS
    
    Args:
        query: Search query text
        index: FAISS index to search
        k: Number of results to return
        df: DataFrame containing test case data
    
    Returns:
        DataFrame with top k similar test cases and their similarity scores
    """
    # Generate embedding for the query
    query_embedding = model.encode([query])
    
    # Search the index
    distances, indices = index.search(query_embedding.astype('float32'), k)
    
    # Create results DataFrame
    results = []
    for i, (dist, idx) in enumerate(zip(distances[0], indices[0])):
        if idx != -1:  # Valid result
            result = df.iloc[idx].copy()
            result['similarity_score'] = 1 / (1 + dist)  # Convert distance to similarity
            result['rank'] = i + 1
            results.append(result)
    
    results_df = pd.DataFrame(results)
    
    # Select relevant columns for display
    if len(results_df) > 0:
        display_columns = ['rank', 'id', 'title', 'category', 'priority', 'similarity_score']
        return results_df[display_columns]
    else:
        return pd.DataFrame()

# Test the search function
test_query = "Test blood pressure monitoring device integration with Bluetooth connectivity"
print(f"🔍 Searching for: '{test_query}'\\n")

results = search_similar_test_cases(test_query, index_flat, k=3)
print("Top 3 similar test cases:")
print(results.to_string(index=False))


🔍 Searching for: 'Test blood pressure monitoring device integration with Bluetooth connectivity'\n
Top 3 similar test cases:
 rank    id                                   title                   category priority  similarity_score
    1 TC002 Blood Pressure Monitor Integration Test Medical Device Integration Critical          0.587196
    2 TC011           Insulin Pump Data Integration Medical Device Integration     High          0.425134
    3 TC004                  HL7 Lab Results Import           Interoperability     High          0.400124


## Step 6: Test Different Query Scenarios

Let's test the system with various healthcare-specific queries to see how well it retrieves relevant test cases:


In [7]:
# Test various query scenarios
test_queries = [
    {
        "query": "HIPAA compliance for patient data security",
        "expected_category": "Compliance",
        "description": "Testing compliance-related search"
    },
    {
        "query": "Emergency room critical patient triage workflow",
        "expected_category": "Clinical Workflow",
        "description": "Testing workflow-related search"
    },
    {
        "query": "Integration with medical devices like insulin pumps and glucose monitors",
        "expected_category": "Medical Device Integration",
        "description": "Testing device integration search"
    },
    {
        "query": "Alert system for drug interactions and allergies",
        "expected_category": "Clinical Decision Support",
        "description": "Testing clinical decision support search"
    },
    {
        "query": "Telehealth video consultation with encrypted communication",
        "expected_category": "Telehealth",
        "description": "Testing telehealth-related search"
    }
]

print("=" * 80)
print("TESTING DIFFERENT QUERY SCENARIOS")
print("=" * 80)

for test_case in test_queries:
    print(f"\n📋 {test_case['description']}")
    print(f"Query: '{test_case['query']}'")
    print(f"Expected Category: {test_case['expected_category']}")
    print("-" * 40)
    
    results = search_similar_test_cases(test_case['query'], index_flat, k=3)
    
    if len(results) > 0:
        for _, row in results.iterrows():
            match_indicator = "✅" if row['category'] == test_case['expected_category'] else "❌"
            print(f"{match_indicator} Rank {row['rank']}: {row['title'][:50]}...")
            print(f"   Category: {row['category']}, Score: {row['similarity_score']:.3f}")
    else:
        print("No results found")
    print()


TESTING DIFFERENT QUERY SCENARIOS

📋 Testing compliance-related search
Query: 'HIPAA compliance for patient data security'
Expected Category: Compliance
----------------------------------------
❌ Rank 1: Patient Registration with Complete Demographics...
   Category: Patient Management, Score: 0.502
❌ Rank 2: Patient Portal Secure Messaging...
   Category: Patient Engagement, Score: 0.475
✅ Rank 3: GDPR Data Export Request...
   Category: Compliance, Score: 0.446


📋 Testing workflow-related search
Query: 'Emergency room critical patient triage workflow'
Expected Category: Clinical Workflow
----------------------------------------
✅ Rank 1: Emergency Department Triage Workflow...
   Category: Clinical Workflow, Score: 0.676
❌ Rank 2: Sepsis Alert Algorithm...
   Category: Clinical Decision Support, Score: 0.504
❌ Rank 3: Patient Portal Secure Messaging...
   Category: Patient Engagement, Score: 0.468


📋 Testing device integration search
Query: 'Integration with medical devices like in

## Step 7: Performance Comparison

Let's compare the performance of different FAISS index types:


In [8]:
import time

def benchmark_index(index, query_texts, k=5):
    """Benchmark search performance of a FAISS index"""
    times = []
    
    for query in query_texts:
        query_embedding = model.encode([query])
        
        start_time = time.time()
        distances, indices = index.search(query_embedding.astype('float32'), k)
        end_time = time.time()
        
        times.append(end_time - start_time)
    
    return {
        'avg_time': np.mean(times) * 1000,  # Convert to milliseconds
        'min_time': np.min(times) * 1000,
        'max_time': np.max(times) * 1000,
        'std_time': np.std(times) * 1000
    }

# Create test queries for benchmarking
benchmark_queries = [
    "Patient registration and demographics",
    "Blood pressure monitoring",
    "GDPR compliance data export",
    "Emergency triage workflow",
    "Medication allergy alerts"
]

# Benchmark each index type
print("🏃 Performance Benchmarking Results")
print("=" * 60)

indices_to_test = {
    'Flat (L2)': index_flat,
    'IVF': index_ivf,
    'HNSW': index_hnsw
}

# Set search parameters for IVF index
index_ivf.nprobe = 3  # Number of clusters to search

benchmark_results = {}
for name, index in indices_to_test.items():
    results = benchmark_index(index, benchmark_queries)
    benchmark_results[name] = results
    
    print(f"\n{name} Index:")
    print(f"  Average search time: {results['avg_time']:.2f} ms")
    print(f"  Min/Max: {results['min_time']:.2f} ms / {results['max_time']:.2f} ms")
    print(f"  Std deviation: {results['std_time']:.2f} ms")

# Create comparison DataFrame
benchmark_df = pd.DataFrame(benchmark_results).T
benchmark_df = benchmark_df.round(2)
print("\n📊 Summary Comparison:")
print(benchmark_df)


🏃 Performance Benchmarking Results

Flat (L2) Index:
  Average search time: 0.03 ms
  Min/Max: 0.03 ms / 0.03 ms
  Std deviation: 0.00 ms

IVF Index:
  Average search time: 0.05 ms
  Min/Max: 0.04 ms / 0.08 ms
  Std deviation: 0.02 ms

HNSW Index:
  Average search time: 0.05 ms
  Min/Max: 0.04 ms / 0.09 ms
  Std deviation: 0.02 ms

📊 Summary Comparison:
           avg_time  min_time  max_time  std_time
Flat (L2)      0.03      0.03      0.03      0.00
IVF            0.05      0.04      0.08      0.02
HNSW           0.05      0.04      0.09      0.02


## Step 8: Save and Load Index

For production use, we need to save and load our FAISS indices:


In [9]:
import os

# Create directory for storing indices
index_dir = "../data/faiss_indices"
os.makedirs(index_dir, exist_ok=True)

# Save the FAISS index
index_path = os.path.join(index_dir, "healthcare_test_cases.index")
faiss.write_index(index_flat, index_path)
print(f"✅ Index saved to: {index_path}")

# Save the metadata (test cases DataFrame)
metadata_path = os.path.join(index_dir, "test_cases_metadata.csv")
df_test_cases.to_csv(metadata_path, index=False)
print(f"✅ Metadata saved to: {metadata_path}")

# Save index configuration
config = {
    "embedding_model": "all-MiniLM-L6-v2",
    "embedding_dimension": dimension,
    "num_test_cases": len(df_test_cases),
    "index_type": "Flat L2",
    "created_date": pd.Timestamp.now().isoformat()
}

config_path = os.path.join(index_dir, "index_config.json")
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)
print(f"✅ Configuration saved to: {config_path}")

print(f"\n📁 Total size of saved files:")
for file in [index_path, metadata_path, config_path]:
    if os.path.exists(file):
        size = os.path.getsize(file) / 1024  # Convert to KB
        print(f"  {os.path.basename(file)}: {size:.2f} KB")


✅ Index saved to: ../data/faiss_indices/healthcare_test_cases.index
✅ Metadata saved to: ../data/faiss_indices/test_cases_metadata.csv
✅ Configuration saved to: ../data/faiss_indices/index_config.json

📁 Total size of saved files:
  healthcare_test_cases.index: 22.54 KB
  test_cases_metadata.csv: 11.99 KB
  index_config.json: 0.17 KB


## Step 9: Load and Verify Saved Index


In [10]:
# Load the saved index
loaded_index = faiss.read_index(index_path)
print(f"✅ Index loaded from: {index_path}")
print(f"   Number of vectors: {loaded_index.ntotal}")

# Load the metadata
loaded_df = pd.read_csv(metadata_path)
# Convert compliance column back to list
loaded_df['compliance'] = loaded_df['compliance'].apply(lambda x: eval(x) if pd.notna(x) else [])
print(f"✅ Metadata loaded: {len(loaded_df)} test cases")

# Load configuration
with open(config_path, 'r') as f:
    loaded_config = json.load(f)
print(f"✅ Configuration loaded:")
for key, value in loaded_config.items():
    print(f"   {key}: {value}")

# Verify the loaded index works correctly
test_query = "Patient data HIPAA compliance"
print(f"\n🧪 Testing loaded index with query: '{test_query}'")

results = search_similar_test_cases(test_query, loaded_index, k=3, df=loaded_df)
print("\nSearch results from loaded index:")
print(results.to_string(index=False))


✅ Index loaded from: ../data/faiss_indices/healthcare_test_cases.index
   Number of vectors: 15
✅ Metadata loaded: 15 test cases
✅ Configuration loaded:
   embedding_model: all-MiniLM-L6-v2
   embedding_dimension: 384
   num_test_cases: 15
   index_type: Flat L2
   created_date: 2025-09-13T17:10:37.808830

🧪 Testing loaded index with query: 'Patient data HIPAA compliance'

Search results from loaded index:
 rank    id                                           title           category priority  similarity_score
    1 TC001 Patient Registration with Complete Demographics Patient Management     High          0.519909
    2 TC014                 Patient Portal Secure Messaging Patient Engagement   Medium          0.470349
    3 TC006                        GDPR Data Export Request         Compliance     High          0.465592


## Step 10: Practical Applications for Test Case Generation

Based on the NASSCOM problem statement, here are practical applications of this FAISS vector database:


In [11]:
class TestCaseGenerator:
    """
    A class that uses FAISS vector search to help generate new test cases
    based on requirements and existing test cases
    """
    
    def __init__(self, index, df_test_cases, model):
        self.index = index
        self.df_test_cases = df_test_cases
        self.model = model
    
    def find_similar_tests(self, requirement: str, k: int = 5):
        """Find existing test cases similar to a new requirement"""
        return search_similar_test_cases(requirement, self.index, k, self.df_test_cases)
    
    def generate_test_template(self, requirement: str):
        """
        Generate a test case template based on similar existing tests
        This simulates what an LLM would do with the retrieved context
        """
        # Find similar test cases
        similar_tests = self.find_similar_tests(requirement, k=3)
        
        if len(similar_tests) == 0:
            return None
        
        # Extract common patterns from similar tests
        top_test = self.df_test_cases[self.df_test_cases['id'] == similar_tests.iloc[0]['id']].iloc[0]
        
        # Create template based on the most similar test
        template = {
            "id": "TC_NEW",
            "title": f"[Generated] {requirement[:50]}...",
            "description": f"Test case for: {requirement}",
            "category": top_test['category'],
            "priority": top_test['priority'],
            "compliance": top_test['compliance'],
            "test_steps": "1. [Step based on requirement]\n2. [Verification step]\n3. [Validation step]",
            "expected_result": "[Expected outcome based on requirement]",
            "based_on": similar_tests['id'].tolist(),
            "similarity_scores": similar_tests['similarity_score'].tolist()
        }
        
        return template
    
    def identify_gaps(self, requirements: List[str], threshold: float = 0.7):
        """
        Identify requirements that don't have similar test cases
        (potential gaps in test coverage)
        """
        gaps = []
        
        for req in requirements:
            results = self.find_similar_tests(req, k=1)
            
            if len(results) == 0 or results.iloc[0]['similarity_score'] < threshold:
                gaps.append({
                    'requirement': req,
                    'best_match_score': results.iloc[0]['similarity_score'] if len(results) > 0 else 0,
                    'best_match_id': results.iloc[0]['id'] if len(results) > 0 else None
                })
        
        return gaps

# Initialize the generator
generator = TestCaseGenerator(index_flat, df_test_cases, model)

# Example 1: Generate test case template for a new requirement
print("=" * 80)
print("EXAMPLE 1: Generate Test Case Template")
print("=" * 80)

new_requirement = "Verify that the system can handle multiple concurrent telemedicine sessions without performance degradation"
template = generator.generate_test_template(new_requirement)

if template:
    print(f"\n📝 New Requirement: {new_requirement}")
    print("\n🤖 Generated Test Case Template:")
    for key, value in template.items():
        if key not in ['based_on', 'similarity_scores']:
            print(f"  {key}: {value}")
    print(f"\n  Based on similar tests: {template['based_on']}")
    print(f"  Similarity scores: {[f'{s:.3f}' for s in template['similarity_scores']]}")


EXAMPLE 1: Generate Test Case Template

📝 New Requirement: Verify that the system can handle multiple concurrent telemedicine sessions without performance degradation

🤖 Generated Test Case Template:
  id: TC_NEW
  title: [Generated] Verify that the system can handle multiple concurr...
  description: Test case for: Verify that the system can handle multiple concurrent telemedicine sessions without performance degradation
  category: Telehealth
  priority: High
  compliance: ['HIPAA', 'State Licensure']
  test_steps: 1. [Step based on requirement]
2. [Verification step]
3. [Validation step]
  expected_result: [Expected outcome based on requirement]

  Based on similar tests: ['TC009', 'TC014', 'TC006']
  Similarity scores: ['0.425', '0.395', '0.389']


In [ ]:
# Example 2: Identify gaps in test coverage
print("\n" + "=" * 80)
print("EXAMPLE 2: Identify Test Coverage Gaps")
print("=" * 80)

new_requirements = [
    "Blockchain-based audit trail for all transactions", 
    "Patient allergy checking system", 
    "AI-powered diagnosis suggestions", 
    "GDPR compliance for EU patients",  
    "Quantum encryption for data security"  
]

print("\n🔍 Analyzing requirements for test coverage gaps...")
gaps = generator.identify_gaps(new_requirements, threshold=0.6)

if gaps:
    print(f"\n⚠️  Found {len(gaps)} potential gaps in test coverage:\n")
    for gap in gaps:
        print(f"  ❌ Requirement: {gap['requirement'][:60]}...")
        print(f"     Best match score: {gap['best_match_score']:.3f}")
        if gap['best_match_id']:
            print(f"     Closest test case: {gap['best_match_id']}")
        print()
else:
    print("\n✅ All requirements have adequate test coverage!")

# Show coverage summary
covered = len(new_requirements) - len(gaps)
coverage_percentage = (covered / len(new_requirements)) * 100
print(f"📊 Test Coverage Summary:")
print(f"   Total requirements: {len(new_requirements)}")
print(f"   Covered: {covered}")
print(f"   Gaps: {len(gaps)}")
print(f"   Coverage: {coverage_percentage:.1f}%")



EXAMPLE 2: Identify Test Coverage Gaps

🔍 Analyzing requirements for test coverage gaps...

⚠️  Found 5 potential gaps in test coverage:

  ❌ Requirement: Blockchain-based audit trail for all transactions...
     Best match score: 0.428
     Closest test case: TC012

  ❌ Requirement: Patient allergy checking system...
     Best match score: 0.517
     Closest test case: TC003

  ❌ Requirement: AI-powered diagnosis suggestions...
     Best match score: 0.443
     Closest test case: TC013

  ❌ Requirement: GDPR compliance for EU patients...
     Best match score: 0.498
     Closest test case: TC006

  ❌ Requirement: Quantum encryption for data security...
     Best match score: 0.385
     Closest test case: TC014

📊 Test Coverage Summary:
   Total requirements: 5
   Covered: 0
   Gaps: 5
   Coverage: 0.0%
